In [2]:
import pandas as pd
import re

In [53]:
# For reading the disctrict_sample_covid.csv file
# df = pd.read_csv('district_sample_covid.csv')
# df = df.drop(['rand_urban_dist', 'rand_non_urb_dist', 'leaid',
#         'nlscdsch','fipst', 'cnty', 'lat', 'lon',
#         'title_one', 'sch_level', 'gslo', 'gshi', 'g01',
#         'g02', 'g03', 'g04', 'g05', 'g06', 'g07', 'g08', 'g09',
#         'g10', 'g11','g12'], axis = 1)
# df = df.rename(columns = {'sch_name' : 'school','lea_name': 'district',
#                           'lcity': 'city','lstate':'state','nmcnty': 'county',
#                           'lstreet':'address','lzip':'zip'})

#For the scrape school list csv
# df = pd.read_csv('scrape_school_list_v1.csv', index_col = 0)
# df = df.drop(['lat','lon','private'], axis = 1)
# df = df.rename(columns = {'sch_name' : 'school','dist_name': 'district',
#                           'lstate':'state','street':'address','sch_level':'level'})
# df = df.astype('object')

#For reading the webscraping schools catalog
# df = pd.read_csv('district_sample_web_catalog - schools.csv')
# df = df.rename(columns = {'sch_name' : 'school','lea_name': 'district',
#                           'lstate':'state','street':'address','sch_level':'level'})
# df = df.astype('object')

In [55]:
#Add leading zeros to zip codes
df['zip']=df['zip'].apply(lambda x: str(x).zfill(5))
#Converts to title case
df = df.apply(lambda x: x.astype(str).str.title() if x.name not in ['address','state'] else x)


In [45]:
#This dict is no longer needed but might be useful for improving speed later
regex_dict = {r"\bh\s*s\b": 'High School',#replaces hs with high school
              r"\bm\s*s\b": 'Middle School',#Same but for ms
              r"\be\s*s\b": 'Elementary School',#same but for es
              r"\be\s*l\b": 'Elementary',#replaces el with elementary school
              r"\bsch\b": "School",#replaces sch with school
              r"\balt\b": "alternative",#replaces alt with alternative
              r"\bhigh$": "High School",#adds school as the final word if name ends with high
              r"\bElementary$": "Elementary School",#adds school as the final word if name ends with middle
              r"\bmiddle$": "Middle School",#adds school as the final word if name ends with elementary
              r"/bacad/b": "Academy",#Replaces acad with academy
             }

In [46]:
#add any more needed regex expressions
regex_list = [(r'\bh\s*s\b', 'High School'),#replaces hs with high school
              (r'\bj[r\s]*h\s*s\b', 'Junior High School'),#replaces jhs or jrhs with junior high school
              (r'\br[r\s]*h\s*s\b', 'Junior High School'),#replaces shs or srhs with senior high school
              (r"\bm\s*s\b", 'Middle School'),#Same but for ms
              (r"\be\s*s\b", 'Elementary School'),#same but for es
              (r"\be\s*l\b", 'Elementary'),#replaces el with elementary school
              (r"\bsch\b", "School"),#replaces sch with school
              (r"\balt\b", "alternative"),#replaces alt with alternative
              (r"\bhigh$", "High School"),#adds school as the final word if name ends with high
              (r"\belementary$", "Elementary School"),#adds school as the final word if name ends with middle
              (r"\bmiddle$", "Middle School"),#adds school as the final word if name ends with elementary
              (r"\bmiddle$", "Middle School"),#adds school as the final word if name ends with charter
              (r"\bacad\b", "Academy"),#Replaces acad with academy
              (r'__+', "_")#Cleans extra white space
             ]

In [47]:
def name_cleaning(text):
    for pattern, replacement in regex_list:
        text = re.sub(pattern,replacement,text,flags = re.I)
    text = re.sub("school.*school",lambda mo: mo.string[6:],text)#Removes the first occurnce of school if there are 2
    return text

In [48]:
#This did not work, but could theoreticcly be faster if that becomes an issue
# def name_cleaning_dict(text):
#     # Create a regular expression  from the dictionary keys
#     regex = re.compile("(%s)" % "|".join(map(re.escape, regex_dict.keys())),flags = re.I)
#     # For each match, look-up corresponding value in dictionary
#     text = regex.sub(lambda mo: regex_dict[mo.string[mo.start():mo.end()]], text)
#     text = re.sub("school.*school",lambda mo: mo.string[6:],text)#Removes the first occurnce of school if there are 2
#     return text

In [56]:
%%timeit -r1 -n1
df['cleaned'] = df['school'].apply(lambda x: name_cleaning(x))

4.75 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Note: Maybe we should filter by the school type so that its just elementary, high and middle
Also question about whether we should add school to ones that end with elementary, middle, or high and then make sure it doesnt have school twice.

In [50]:
df.sample(10)['cleaned']

7          Frederick Douglass High School
73                Ridgeview Middle School
105                 Westville High School
8      Booker T. Washington Middle School
109            Albert Powell Continuation
82               Jordan Elementary School
20         Richmond Heights Middle School
9         Fountain Hill Elementary School
65          Goldenview Colony School - 04
13                 Broughal Middle School
Name: cleaned, dtype: object

In [57]:
#df.to_csv('district_sample_web_catalog - schools_cleaned.csv')